In [1]:
import pandas as pd
import re
from evaluate import load

# 데이터 로드
submission = pd.read_csv('./prediction/submit_solar_v3.csv')
dev_df = pd.read_csv('./data/dev.csv')

print(f"제출 파일: {len(submission)}개")
print(f"Dev 파일: {len(dev_df)}개")

# ROUGE 메트릭
rouge = load("rouge")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


제출 파일: 499개
Dev 파일: 499개


## 1단계: 후처리 함수 정의

v1 ~ v6까지의 후처리 함수를 정의합니다.

In [2]:
def postprocess_v1(summary: str) -> str:
    """기본 후처리: 따옴표 + 공백 정리"""
    summary = summary.strip('""''""')
    summary = re.sub(r'\s+', ' ', summary).strip()
    return summary


def postprocess_v2(summary: str) -> str:
    """중급 후처리: v1 + 불완전한 문장 제거"""
    summary = postprocess_v1(summary)
    sentences = re.split(r'([.!?]\s+)', summary)
    complete_text = ""
    for i in range(0, len(sentences) - 1, 2):
        if i + 1 < len(sentences):
            complete_text += sentences[i] + sentences[i + 1]
    if sentences and sentences[-1].strip() and sentences[-1].strip()[-1] in '.!?다요':
        complete_text += sentences[-1]
    return complete_text.strip()


def postprocess_v3(summary: str, max_sentences: int = 3) -> str:
    """고급 후처리: v2 + 문장 수 제한"""
    summary = postprocess_v2(summary)
    sentences = re.split(r'(?<=[.!?])\s+', summary)
    if len(sentences) > max_sentences:
        sentences = sentences[:max_sentences]
    return ' '.join(sentences).strip()


def postprocess_v4(summary: str, max_sentences: int = 3) -> str:
    """최종 후처리: v3 + 특수토큰 정규화"""
    summary = postprocess_v3(summary, max_sentences)
    summary = re.sub(r'#Person(\d+)#', r'#Person\1#', summary)
    summary = re.sub(r'#PhoneNumber#', '#PhoneNumber#', summary)
    summary = re.sub(r'\s+', ' ', summary).strip()
    return summary


def postprocess_v5_aggressive(summary: str, max_sentences: int = 2) -> str:
    """공격적 후처리: v4 + 접속사 제거 + 표현 통일"""
    summary = postprocess_v4(summary, max_sentences=max_sentences)
    summary = re.sub(r'(?<=\.)\s(그리고|또한|하지만|그러나|그런데)\s+', '. ', summary)
    summary = re.sub(r'라고\s+말합니다', '라고 합니다', summary)
    summary = re.sub(r'이라고\s+말합니다', '이라고 합니다', summary)
    summary = re.sub(r'\s+', ' ', summary).strip()
    return summary


def postprocess_v6_ultra(summary: str) -> str:
    """초공격적 후처리: 핵심 정보만 추출"""
    summary = postprocess_v5_aggressive(summary, max_sentences=2)
    sentences = re.split(r'(?<=[.!?])\s+', summary)
    if len(sentences) > 2:
        sentences = sentences[:2]
    elif len(sentences) == 1 and len(sentences[0].split(',')) > 4:
        parts = sentences[0].split(',')
        sentences = [','.join(parts[:3]) + '.']
    return ' '.join(sentences).strip()


print("✅ 모든 후처리 함수 정의 완료")

✅ 모든 후처리 함수 정의 완료


## 2단계: 현재 성공 버전 로드

이미 제출하여 51.94점을 획득한 v3_postprocessed 버전을 로드합니다.

In [3]:
# 원본과 현재 베스트 버전 로드
submission_orig = pd.read_csv('./prediction/submit_solar_v3.csv')
submission_current_best = pd.read_csv('./prediction/submit_solar_v3_postprocessed.csv')

print("📊 현재 상태:")
print(f"  - 원본: submit_solar_v3.csv (51.7703점)")
print(f"  - 현재 베스트: submit_solar_v3_postprocessed.csv (51.9393점)")
print(f"  - 개선: +0.169점 ✅")

# 길이 비교
orig_len = submission_orig['summary'].apply(lambda x: len(str(x).split())).mean()
best_len = submission_current_best['summary'].apply(lambda x: len(str(x).split())).mean()

print(f"\n📏 평균 길이:")
print(f"  - 원본: {orig_len:.1f} 단어")
print(f"  - 베스트: {best_len:.1f} 단어")

📊 현재 상태:
  - 원본: submit_solar_v3.csv (51.7703점)
  - 현재 베스트: submit_solar_v3_postprocessed.csv (51.9393점)
  - 개선: +0.169점 ✅

📏 평균 길이:
  - 원본: 16.1 단어
  - 베스트: 15.6 단어


## 3단계: 추가 개선 버전 생성

ROUGE-2 최적화를 위해 더 공격적인 후처리를 시도합니다.

In [4]:
print("🔬 새로운 후처리 버전 생성 중...\n")

# 새 버전들 생성
v5_2sent = submission_orig['summary'].apply(lambda x: postprocess_v5_aggressive(x, 2)).tolist()
v5_1sent = submission_orig['summary'].apply(lambda x: postprocess_v5_aggressive(x, 1)).tolist()
v6_ultra = submission_orig['summary'].apply(postprocess_v6_ultra).tolist()

# 통계
print("📊 버전별 길이 비교:\n")
print(f"  [현재 베스트 (3문장)]     평균: {best_len:5.1f} 단어 (리더보드: 51.94)")

v5_2_len = sum(len(s.split()) for s in v5_2sent) / len(v5_2sent)
print(f"  [v5 공격적 (2문장)]     평균: {v5_2_len:5.1f} 단어 (예상: 51.95~52.05)")

v5_1_len = sum(len(s.split()) for s in v5_1sent) / len(v5_1sent)
print(f"  [v5 공격적 (1문장)]     평균: {v5_1_len:5.1f} 단어 (예상: 51.90~52.10)")

v6_len = sum(len(s.split()) for s in v6_ultra) / len(v6_ultra)
print(f"  [v6 초공격적]          평균: {v6_len:5.1f} 단어 (예상: 51.85~52.15)")

print("\n✅ 버전 생성 완료")

🔬 새로운 후처리 버전 생성 중...

📊 버전별 길이 비교:

  [현재 베스트 (3문장)]     평균:  15.6 단어 (리더보드: 51.94)
  [v5 공격적 (2문장)]     평균:  14.4 단어 (예상: 51.95~52.05)
  [v5 공격적 (1문장)]     평균:   9.7 단어 (예상: 51.90~52.10)
  [v6 초공격적]          평균:  14.3 단어 (예상: 51.85~52.15)

✅ 버전 생성 완료


## 4단계: 샘플 비교

주요 케이스에서 각 버전이 어떻게 다른지 확인합니다.

In [5]:
test_cases = [0, 5, 41]

print("🔍 주요 케이스 비교:\n")
for idx in test_cases:
    orig = submission_orig.iloc[idx]['summary']
    current = submission_current_best.iloc[idx]['summary']
    v5_2 = v5_2sent[idx]
    
    print(f"\n{'='*80}")
    print(f"[{submission_orig.iloc[idx]['fname']}]")
    print(f"{'='*80}")
    
    print(f"\n[원본] ({len(orig.split())} 단어)")
    print(orig[:150] + "..." if len(orig) > 150 else orig)
    
    print(f"\n[현재 베스트 51.94] ({len(current.split())} 단어)")
    print(current)
    
    print(f"\n[v5 공격적 2문장] ({len(v5_2.split())} 단어)")
    print(v5_2)

🔍 주요 케이스 비교:


[test_0]

[원본] (34 단어)
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다. 이 정책은 모든 통신에 적용되며, 즉시 메시지 사용은 금지됩니다. Ms. Dawson은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용한다고 말하지만, #Person1...

[현재 베스트 51.94] (20 단어)
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다. 이 정책은 모든 통신에 적용되며, 즉시 메시지 사용은 금지됩니다.

[v5 공격적 2문장] (11 단어)
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다.

[test_5]

[원본] (26 단어)
#Person1#은 회사에서 일하는 것을 그만두고 자신의 사업을 시작하기로 결정했습니다. #Person2#는 #Person1#에게 사업 계획서를 작성하는 방법을 설명합니다. #Person1#은 사업 시작의 번거로움을 피하기 위해 예전 일을 계속하기로 결정합니다.

[현재 베스트 51.94] (26 단어)
#Person1#은 회사에서 일하는 것을 그만두고 자신의 사업을 시작하기로 결정했습니다. #Person2#는 #Person1#에게 사업 계획서를 작성하는 방법을 설명합니다. #Person1#은 사업 시작의 번거로움을 피하기 위해 예전 일을 계속하기로 결정합니다.

[v5 공격적 2문장] (16 단어)
#Person1#은 회사에서 일하는 것을 그만두고 자신의 사업을 시작하기로 결정했습니다. #Person2#는 #Person1#에게 사업 계획서를 작성하는 방법을 설명합니다.

[test_41]

[원본] (33 단어)
#Person1#은 박싱 데이 세일에 갈 계획이라 #Person2#에게 새 옷에 대해 의견을 묻고 있습니다. #Person2#는 게임을 보고 싶어 하지만, #Person1#은 계속 옷에 대해 이야기하고 있습니다. #Person2

## 5단계: 제출 파일 생성

3개의 새 제출 파일을 생성합니다.

In [6]:
print("📝 제출 파일 생성 중...\n")

# 파일 1: v5 공격적 2문장 (권장)
df_v5_2 = submission_orig[['fname']].copy()
df_v5_2['summary'] = v5_2sent
path_v5_2 = './prediction/submit_solar_v4_aggressive_2sent.csv'
df_v5_2.to_csv(path_v5_2, index=False)

print(f"✅ [1] submit_solar_v4_aggressive_2sent.csv")
print(f"   평균 길이: {v5_2_len:.1f} 단어")
print(f"   전략: 3문장 → 2문장 + 접속사 제거")
print(f"   예상 점수: 51.95~52.05\n")

# 파일 2: v5 공격적 1문장
df_v5_1 = submission_orig[['fname']].copy()
df_v5_1['summary'] = v5_1sent
path_v5_1 = './prediction/submit_solar_v4_aggressive_1sent.csv'
df_v5_1.to_csv(path_v5_1, index=False)

print(f"✅ [2] submit_solar_v4_aggressive_1sent.csv")
print(f"   평균 길이: {v5_1_len:.1f} 단어")
print(f"   전략: 1문장으로 극한 압축")
print(f"   예상 점수: 51.90~52.10\n")

# 파일 3: v6 초공격적
df_v6 = submission_orig[['fname']].copy()
df_v6['summary'] = v6_ultra
path_v6 = './prediction/submit_solar_v5_ultra.csv'
df_v6.to_csv(path_v6, index=False)

print(f"✅ [3] submit_solar_v5_ultra.csv")
print(f"   평균 길이: {v6_len:.1f} 단어")
print(f"   전략: 핵심 정보만 극도로 압축")
print(f"   예상 점수: 51.85~52.15")

📝 제출 파일 생성 중...

✅ [1] submit_solar_v4_aggressive_2sent.csv
   평균 길이: 14.4 단어
   전략: 3문장 → 2문장 + 접속사 제거
   예상 점수: 51.95~52.05

✅ [2] submit_solar_v4_aggressive_1sent.csv
   평균 길이: 9.7 단어
   전략: 1문장으로 극한 압축
   예상 점수: 51.90~52.10

✅ [3] submit_solar_v5_ultra.csv
   평균 길이: 14.3 단어
   전략: 핵심 정보만 극도로 압축
   예상 점수: 51.85~52.15


## 6단계: 제출 가이드

추천 제출 순서와 방법

In [7]:
print("\n" + "="*80)
print("📋 리더보드 제출 가이드")
print("="*80)

print("\n🥇 [1순위] submit_solar_v4_aggressive_2sent.csv")
print("   상황:")
print("     - 점수 올라면 (52.0+): 추가 개선 가능! 다음 파일 시도")
print("     - 점수 비슷 (51.92~51.96): 현재 버전 유지")
print("     - 점수 내려가면: 현재 51.94점 유지\n")

print("🥈 [2순위] submit_solar_v4_aggressive_1sent.csv")
print("   조건: v5_2sent가 51.94보다 높을 경우만 시도")
print("   위험도: 높음 (1문장만 사용)\n")

print("🥉 [3순위] submit_solar_v5_ultra.csv")
print("   조건: 모든 것이 안 될 경우의 최후 수단")
print("   위험도: 매우 높음\n")

print("="*80)
print("✅ 현재 상태: 51.94점 (이미 성공!)")
print("목표: 52.0점 이상 달성")
print("="*80)


📋 리더보드 제출 가이드

🥇 [1순위] submit_solar_v4_aggressive_2sent.csv
   상황:
     - 점수 올라면 (52.0+): 추가 개선 가능! 다음 파일 시도
     - 점수 비슷 (51.92~51.96): 현재 버전 유지
     - 점수 내려가면: 현재 51.94점 유지

🥈 [2순위] submit_solar_v4_aggressive_1sent.csv
   조건: v5_2sent가 51.94보다 높을 경우만 시도
   위험도: 높음 (1문장만 사용)

🥉 [3순위] submit_solar_v5_ultra.csv
   조건: 모든 것이 안 될 경우의 최후 수단
   위험도: 매우 높음

✅ 현재 상태: 51.94점 (이미 성공!)
목표: 52.0점 이상 달성
